In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import *
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

In [14]:
path_exit = "/home/ale1726/proyects/datalake/clientes/data/productos/SIAG/agrupados"
date_now = datetime.now().strftime("%d_%m_%Y") 


In [2]:
df_productos = pd.read_csv('/home/ale1726/proyects/datalake/clientes/data/productos/SIAG/data/26_02_2025/productos_clientes_SIAG.dat', low_memory=True)

In [4]:
df_productos.columns

Index(['NUMERO_DE_GARANTIA', 'NUMERO_CLIENTE', 'CLIENTE', 'DESC_CORTA',
       'SUBPRODUCTO', 'FECHA_REGISTRO_GARANTIA', 'MONTO_CREDITO',
       'PORCENTAJE_GARANTIZADO', 'MONTO_GARANTIZADO', 'TASA_INTERES',
       'PORCENTAJE_COMISION', 'GANANCIA', 'AIO', 'MES', 'DIA'],
      dtype='object')

In [22]:
agrupado_año_garantias = (
    df_productos.groupby("AIO").agg(
        NUMERO_INTERMEDIARIOS_ANUAL = ("NUMERO_CLIENTE", pd.Series.nunique),
        GARANTIAS_X_AÑO = ("NUMERO_CLIENTE", "count"),
        SUMA_ANUAL_MONTO_GARANTIZADO = ("MONTO_GARANTIZADO", lambda x: round(x.sum())),
        PROMEDIO_ANUAL_MONTO_GARANTIZADO_X_GARANTIA = ("MONTO_GARANTIZADO", lambda x: round(x.mean())),
        TOTAL_ANUAL_GANANCIA = ("GANANCIA", "sum"),
        PROMEDIO_ANUAL_COMISION = ("PORCENTAJE_COMISION", lambda x: round(x.mean(), 4))
    )
    .assign(
        PROMEDIO_ANUAL_GARANTIAS_X_INTERMEDARIO = 
        lambda d: (d["GARANTIAS_X_AÑO"] / d["NUMERO_INTERMEDIARIOS_ANUAL"]).round(2)
    )
    .reset_index()
).sort_values(["AIO"], ascending=True)

In [24]:
agrupado_año_garantias

,AIO,NUMERO_INTERMEDIARIOS_ANUAL,GARANTIAS_X_AÑO,SUMA_ANUAL_MONTO_GARANTIZADO,PROMEDIO_ANUAL_MONTO_GARANTIZADO_X_GARANTIA,TOTAL_ANUAL_GANANCIA,PROMEDIO_ANUAL_COMISION,PROMEDIO_ANUAL_GARANTIAS_X_INTERMEDARIO
0,2002,1,600,143693810,239490,0,0.0000,600.00
1,2003,1,232,113472418,489105,0,0.0000,232.00
2,2004,1,332,214108320,644905,0,0.0000,332.00
3,2005,2,3634,412333355,113465,0,0.0000,1817.00
4,2006,2,15019,920508133,61290,49228386,0.7179,7509.50
5,2007,3,10974,834027329,76000,49081621,1.0425,3658.00
6,2008,11,7411,6151528669,830054,18273045,0.4353,673.73
7,2009,22,119514,12834023600,107385,20236331,0.0212,5432.45
8,2010,31,459595,1190184993,2590,68311827,0.0317,14825.65
9,2011,31,89734,1528202771,17030,263722571,0.5346,2894.65


In [15]:
agrupado_año_garantias.to_csv(os.path.join(path_exit,"agrupado_año_garantias.csv"),index=False)

In [16]:
agrupado_año_mes_garantias = (
    df_productos.groupby(["AIO","MES"]).agg(
        NUMERO_INTERMEDIARIOS_ANUAL = ("NUMERO_CLIENTE", pd.Series.nunique),
        GARANTIAS_X_AÑO = ("NUMERO_CLIENTE", "count"),
        SUMA_ANUAL_MONTO_GARANTIZADO = ("MONTO_GARANTIZADO", lambda x: round(x.sum())),
        PROMEDIO_ANUAL_MONTO_GARANTIZADO_X_GARANTIA = ("MONTO_GARANTIZADO", lambda x: round(x.mean())),
        TOTAL_ANUAL_GANANCIA = ("GANANCIA", "sum"),
        PROMEDIO_ANUAL_COMISION = ("PORCENTAJE_COMISION", lambda x: round(x.mean(), 4))
    )
    .assign(
        PROMEDIO_ANUAL_GARANTIAS_X_INTERMEDARIO = 
        lambda d: (d["GARANTIAS_X_AÑO"] / d["NUMERO_INTERMEDIARIOS_ANUAL"]).round(2)
    )
    .reset_index()
)

In [17]:
agrupado_año_mes_garantias.to_csv(os.path.join(path_exit,"agrupado_año_mes_garantias.csv"),index=False)

In [ ]:
SELECT 
			CLIENTE,
			SUM(GANANCIA) AS COMISIONES_PAGADAS_X_INTERMEDIARIO,
			COUNT(*) NUMERO_GARANTIAS_ACTIVAS_X_INTERMEDIARIO,
			ROUND(AVG(PORCENTAJE_COMISION),2) AS PROMEDIO_COMISION_X_INTER,
			ROUND(SUM(MONTO_GARANTIZADO)) AS TOTAL_MONTO_GARANTIZADOS
			FROM INFORMACION_GARANTIA
			GROUP BY CLIENTE

In [ ]:
"AÑO",
CLIENTE,
COUNT(NUMERO_DE_GARANTIA) AS NUMERO_DE_GARANTIAS,
SUM(MONTO_GARANTIZADO) AS TOTAL_MONTO_GARANTIZADO,
ROUND(AVG(MONTO_GARANTIZADO)) PROMEDIO_MONTO_GARANTIZADO,
ROUND(AVG(PORCENTAJE_COMISION),4) PROMEDIO_COMISION,
SUM(GANANCIA) AS PAGO_COMISION_CLIENTE


In [26]:
df_productos.columns

Index(['NUMERO_DE_GARANTIA', 'NUMERO_CLIENTE', 'CLIENTE', 'DESC_CORTA',
       'SUBPRODUCTO', 'FECHA_REGISTRO_GARANTIA', 'MONTO_CREDITO',
       'PORCENTAJE_GARANTIZADO', 'MONTO_GARANTIZADO', 'TASA_INTERES',
       'PORCENTAJE_COMISION', 'GANANCIA', 'AIO', 'MES', 'DIA'],
      dtype='object')

In [ ]:
agrupado = (
    df_productos.groupby(["AIO","CLIENTE"]).agg(
        NUMERO_DE_GARANTIAS = ("NUMERO_CLIENTE", pd.Series.nunique),
        TOTAL_MONTO_GARANTIZADO = ("NUMERO_CLIENTE", "count"),
        PROMEDIO_MONTO_GARANTIZADO = ("MONTO_GARANTIZADO", lambda x: round(x.sum())),
        PROMEDIO_COMISION = ("MONTO_GARANTIZADO", lambda x: round(x.mean())),
        PAGO_COMISION_CLIENTE = ("GANANCIA", "sum"),
    )
    .assign(
        PROMEDIO_ANUAL_GARANTIAS_X_INTERMEDARIO = 
        lambda d: (d["GARANTIAS_X_AÑO"] / d["NUMERO_INTERMEDIARIOS_ANUAL"]).round(2)
    )
    .reset_index()
)